Importing necessary libraries

In [6]:
if __name__ == '__main__':
    import torch
    import torchvision
    import torchvision.transforms as transforms
    from torch.utils.data import DataLoader
    from torchvision import datasets
    from torchvision.transforms import ToTensor
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim
    import matplotlib.pyplot as plt
    import numpy as np
    from collections import Counter
    

     # Define transformations (resize and convert to tensor)
    transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
    ])

    # Define dataset directories
    train_dir = "../data/Training"
    test_dir = "../data/Testing"
    valid_dir = "../data/Validation"

    # Load datasets
    train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
    valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

    # Print class-to-index mapping
    print("Class mapping:", train_dataset.class_to_idx)

    # Check a batch of images and labels
    images, labels = next(iter(train_loader))
    print(f"Batch image shape: {images.shape}")
    print(f"Batch labels: {labels}")

    # Checking the label counts
    label_counts = Counter(train_dataset.targets)
    print("Training class distribution:", label_counts)

    label_counts = Counter(test_dataset.targets)
    print("Testing class distribution:", label_counts)  

    label_counts = Counter(valid_dataset.targets)
    print("Validation class distribution:", label_counts)  
    

Class mapping: {'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}
Batch image shape: torch.Size([32, 3, 224, 224])
Batch labels: tensor([0, 0, 1, 3, 2, 3, 0, 0, 1, 1, 0, 3, 1, 2, 1, 0, 2, 2, 3, 2, 1, 2, 2, 2,
        1, 3, 2, 0, 2, 3, 0, 1])
Training class distribution: Counter({3: 827, 0: 826, 1: 822, 2: 395})
Testing class distribution: Counter({1: 58, 0: 50, 3: 37, 2: 27})
Validation class distribution: Counter({1: 57, 0: 50, 3: 37, 2: 26})


In [13]:
class BrainTumorCNN(nn.Module):
    def __init__(self, num_classes):
        super(BrainTumorCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Get number of classes
num_classes = len(train_dataset.classes)

# Initialize the model
model = BrainTumorCNN(num_classes=num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
weights = torch.tensor([2.0, 1.0, 1.0, 1.0])
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Train for 10 epochs
train_model(model, train_loader, criterion, optimizer, epochs=10)


KeyboardInterrupt: 

In [9]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Evaluate the model
evaluate_model(model, test_loader)

Test Accuracy: 70.35%


In [ ]:
import torch

def calculate_recall_without_sklearn(model, data_loader, num_classes):
    model.eval()
    true_positives = torch.zeros(num_classes)
    false_negatives = torch.zeros(num_classes)
    
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for i in range(num_classes):
                true_positives[i] += ((predicted == i) & (labels == i)).sum().item()
                false_negatives[i] += ((predicted != i) & (labels == i)).sum().item()
    
    # Compute recall per class
    recall = true_positives / (true_positives + false_negatives)
    recall[torch.isnan(recall)] = 0  # Handle division by zero cases

    # Convert to dictionary
    recall_dict = {f"Class {i}": recall[i].item() for i in range(num_classes)}

    return recall_dict

# Example usage:
recall_rates = calculate_recall_without_sklearn(model, test_loader, num_classes)
print("Recall per class:", recall_rates)


Recall per class: {'Class 0': 0.20000000298023224, 'Class 1': 0.982758641242981, 'Class 2': 1.0, 'Class 3': 0.7297297120094299}


In [12]:
def compute_confusion_matrix(model, data_loader, num_classes):
    model.eval()
    confusion_matrix = torch.zeros(num_classes, num_classes, dtype=torch.int64)

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t, p] += 1  # Increment the matrix count

    return confusion_matrix

# Example usage:
conf_matrix = compute_confusion_matrix(model, test_loader, num_classes)

def print_confusion_matrix(conf_matrix, class_names):
    print("\nConfusion Matrix:")
    print(" " * 10, end="")  # Formatting for column headers
    print("  ".join(f"{cls[:5]:>5}" for cls in class_names))  # Print class names as column headers

    for i, row in enumerate(conf_matrix):
        print(f"{class_names[i][:5]:>10} ", end="")  # Row labels
        print("  ".join(f"{val:5}" for val in row.tolist()))  # Print matrix row values

# Example usage:
print_confusion_matrix(conf_matrix, train_dataset.classes)


Confusion Matrix:
          gliom  menin  no_tu  pitui
     gliom    10     14     25      1
     menin     1     57      0      0
     no_tu     0      0     27      0
     pitui     2      4      4     27
None
